In [ ]:
#Load libs

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from tysserand import tysserand as ty
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 
from PIL import Image, ImageOps
import fcsparser

#set up working dir

data_dir = Path("/home/mouneem/Projects/tysserand/tysserand/Imagin_LungPredict/")
path = str("/home/mouneem/Projects/tysserand/tysserand/Imagin_LungPredict/")


In [ ]:
# FCS TO CSV
fileimg = "C1v1/NVA_21-003.IMMCORE.C1v1_15T008187-08-ImvessC1-4518.tif"

FCSfile = 'NVA_21-003.IMMCORE.C1v1_15T008187-08-ImvessC1-4518.czi_79677_ObjData.fcs'
meta, data = fcsparser.parse(path+"fcs/"+FCSfile, reformat_meta=True)
type(data)
#df1 = df[['a', 'b']] // df1 = df.iloc[:, 0:2] 

data["x"] = (data["XMax"] + data["XMin"] ) / 2
data["y"] = (data["YMax"] + data["YMin"] ) / 2

col_is_marker = []
for col in list(data.columns):
    if col.count('+') + col.count('-') > 1:
        col_is_marker.append(True)
    else:
        col_is_marker.append(False)

print(len(col_is_marker))

marker = data.loc[:, col_is_marker]

classes = marker.apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
data["class"] = pd.factorize(classes)[0] 

nodes = data[['x','y',"class"]]
nodes.to_csv(path + "csv/"+FCSfile )

In [ ]:
ratio = 10
nodes = nodes.sample(n = round(len(nodes) / ratio) )
coords = nodes.loc[:,['x','y']].values

print('nodes:', len(nodes))
print('coords:', len(coords))

#load image
img = plt.imread(data_dir / fileimg)
#img = ImageOps.grayscale(img) # GRAYSCALE IMAGE


In [ ]:
#make colors

class_colors = ['#F85446', '#813FE0', '#52DEF7', '#62E03F', '#FFCE36', '#FA6C17','#D805E3', '#0155FA', '#0BE35B', '#FFF117', '#DDFF7D', '#78E3C8']
classes = list(nodes['class'].unique())
print(classes)

dico_col = {}
for i in range(len(classes)):
    marker = classes[i]
    dico_col[marker] = class_colors[i]


colors = []
for cl in nodes['class']:
    colors.append(dico_col[cl])


In [ ]:
# Adjust size of image  

# nodes
min_x = min(coords[:, 0])
max_x = max(coords[:, 0])
nodes_width = max_x - min_x   

min_y = min(coords[:, 1])
max_y = max(coords[:, 1])
nodes_height = max_y - min_y   

# img
img_width = img.shape[0]
img_height = img.shape[1]

ratio_x = img_width / nodes_width
ratio_y = img_height / nodes_height
ratio = 1
print('ratio x', ratio_x, ' \n'
        'ratio y', ratio_y)

#img = cv2.flip(img, 0)
coords[:,0]  = (coords[:,0] * ratio_x ).round()
coords[:,1]  = (coords[:,1] * ratio_x).round()
print('scaled UP')

## TANSITION TO 0
coords[:,0] = coords[:,0] - min(coords[:,0])
coords[:,1] = coords[:,1] - min(coords[:,1])


In [ ]:
plt.scatter(coords[:,0] , coords[:,1] , c=colors,  zorder=10, s = 1)

In [ ]:

fig, ax = ty.showim(img, figsize=(150, 150))
col_nodes = colors
ax.scatter(coords[:,0] , coords[:,1] , c=col_nodes,  zorder=10)

# TYSSERAND

In [ ]:
# superimpose network to mIF image

fig, ax = ty.showim(img, figsize=(100, 100))

pairs = ty.build_delaunay(coords)
#distances = ty.distance_neighbors(coords, pairs)
    
ty.plot_network(coords, pairs, col_nodes=colors, col_edges='w', ax=ax)